In [230]:
# Required Modules
import pandas as pd
import sqlite3
from sqlite3 import Error
import os

# Identify database path
db_path = os.path.abspath('./sqlite.db')

# Connect to SQLite Database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

In [231]:
# FL Zip Codes from Census data for reference
fl_census_data = pd.read_csv('Resources/ACSST5Y2019.S1901_data_with_overlays_2021-05-24T165952.csv', header=1)
fl_codes = []

for i,j in fl_census_data.iterrows():
    fl_codes.append(j['Geographic Area Name'][6:11])

fl_zipcodes = pd.DataFrame(fl_codes)
fl_zipcodes = fl_zipcodes.rename(columns={0:'zipcode'})

# Drop table if it exist
cursor.execute("DROP TABLE IF EXISTS zipcode")

# Create table
sql_table.to_sql('zipcode', conn, if_exists='replace', index=False)

In [232]:
tables_ls = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
tables_ls

,name
0,mobility
1,household_income
2,mobility_slim
3,grades
4,fha_loans
5,zipcode


In [233]:
fha_loans = pd.read_sql("SELECT * FROM fha_loans", conn)
mobility_slim = pd.read_sql("SELECT * FROM mobility_slim", conn)
mobility = pd.read_sql("SELECT * FROM mobility", conn)
household_income = pd.read_sql("SELECT * FROM household_income", conn)
grades = pd.read_sql("SELECT * FROM grades", conn)

In [234]:
all_data = pd.read_sql('''SELECT    zip.zipcode, 
                                    fha.year, 
                                    fha.month, 
                                    fha.fha, 
                                    mob.mobility_rate, 
                                    inc.B19013001 AS household_income, 
                                    g.percent_total_points AS school_rating, 
                                    g.letter_grade AS school_grade
                                FROM zipcode AS zip
                                LEFT JOIN fha_loans AS fha ON zip.zipcode = fha.zipcode
                                LEFT JOIN mobility_slim AS mob ON zip.zipcode = mob.name
                                LEFT JOIN household_income AS inc ON zip.zipcode = inc.name
                                LEFT JOIN grades AS g ON zip.zipcode = g.zip
                        ''', conn)

In [235]:
all_data.head()

,zipcode,year,month,fha,mobility_rate,household_income,school_rating,school_grade
0,32003,2019,1,7,0.182327,94154.0,None,None
1,32003,2020,1,3,0.182327,94154.0,None,None
2,32003,2019,2,2,0.182327,94154.0,None,None
3,32003,2020,2,5,0.182327,94154.0,None,None
4,32003,2019,3,7,0.182327,94154.0,None,None


In [236]:
# Close connection to SQLite Database
if conn:
    conn.close()

In [237]:
print(f"FHA Loans: {fha_loans[['zipcode','month','year']].nunique()}")
print(f"Mobility Slim: {mobility_slim['name'].nunique()}")
print(f"Mobility: {mobility['name'].nunique()}")
print(f"Grades: {grades['zip'].nunique()}")
print(f"Income: {household_income['name'].nunique()}")

FHA Loans: zipcode    909
month       12
year         2
dtype: int64
Mobility Slim: 977
Mobility: 977
Grades: 477
Income: 947


In [238]:
# FL Zip Codes from Census data for reference
fl_census_data = pd.read_csv('Resources/ACSST5Y2019.S1901_data_with_overlays_2021-05-24T165952.csv', header=1)
fl_codes = []

for i,j in fl_census_data.iterrows():
    fl_codes.append(j['Geographic Area Name'][6:11])

len(fl_codes)

984